In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline


In [50]:
spark = SparkSession.builder \
    .appName("DiabetesReadmissionPrediction") \
    .getOrCreate()


In [51]:
# Đọc file CSV từ đường dẫn
file_path = 'diabetic_data.csv'

# Đọc dữ liệu CSV
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Hiển thị một số dòng đầu của dữ liệu
df.show(5)


+------------+-----------+---------------+------+-------+------+-----------------+------------------------+-------------------+----------------+----------+--------------------+------------------+--------------+---------------+-----------------+----------------+----------------+------+------+------+----------------+-------------+---------+---------+-----------+-----------+--------------+-----------+-------------+---------+---------+-----------+------------+-------------+--------+--------+------------+----------+-------+-----------+-------+-------------------+-------------------+------------------------+-----------------------+----------------------+------+-----------+----------+
|encounter_id|patient_nbr|           race|gender|    age|weight|admission_type_id|discharge_disposition_id|admission_source_id|time_in_hospital|payer_code|   medical_specialty|num_lab_procedures|num_procedures|num_medications|number_outpatient|number_emergency|number_inpatient|diag_1|diag_2|diag_3|number_diagnos

In [52]:
# Loại bỏ các cột không cần thiết
columns_to_drop = ['encounter_id', 'patient_nbr', 'payer_code', 'weight']
df = df.drop(*columns_to_drop)

# Thay thế các giá trị '?' bằng 'unknown'
df = df.replace('?', 'unknown')

# Mã hóa cột 'readmitted' (NO -> 0, >30 -> 1, <30 -> 2)
df = df.withColumn('readmitted', when(col('readmitted') == 'NO', 0)
                   .when(col('readmitted') == '>30', 1)
                   .when(col('readmitted') == '<30', 2))

# Xác định các cột cần mã hóa
categorical_columns = ['race', 'gender', 'age', 'max_glu_serum', 'A1Cresult', 
                       'medical_specialty', 'change', 'diabetesMed']

# Sử dụng StringIndexer và OneHotEncoder cho các cột phân loại
indexers = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in categorical_columns]
encoders = [OneHotEncoder(inputCol=col + "_index", outputCol=col + "_encoded") for col in categorical_columns]

# Tạo feature vector từ tất cả các cột
feature_columns = [col + "_encoded" for col in categorical_columns] + [
    'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 
    'time_in_hospital', 'num_lab_procedures', 'num_procedures', 
    'num_medications', 'number_outpatient', 'number_emergency', 
    'number_inpatient', 'number_diagnoses'
]

# Tạo vector features từ các cột đầu vào
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")



In [53]:
# Khởi tạo mô hình Random Forest
rf_classifier = RandomForestClassifier(labelCol="readmitted", featuresCol="features", numTrees=100)

# Tạo pipeline để nối các bước
pipeline = Pipeline(stages=indexers + encoders + [assembler, rf_classifier])


In [ ]:
# Tách dữ liệu thành 80% train và 20% test
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

train_data = data.groupby('readmitted', group_keys=False).apply(lambda x: x.sample(frac=0.8, random_state=42))
test_data = data.drop(train_data.index)


In [14]:
# Huấn luyện pipeline trên tập train
model = pipeline.fit(train_data)


In [15]:
# Thực hiện dự đoán trên tập test
predictions = model.transform(test_data)

# Hiển thị 5 dự đoán đầu tiên
predictions.select("readmitted", "prediction", "probability").show(5)


+----------+----------+--------------------+
|readmitted|prediction|         probability|
+----------+----------+--------------------+
|         1|       0.0|[0.63701092874192...|
|         1|       0.0|[0.51687346709739...|
|         0|       0.0|[0.62197536982776...|
|         1|       0.0|[0.51424233877733...|
|         1|       0.0|[0.60286208789552...|
+----------+----------+--------------------+
only showing top 5 rows

